<a href="https://colab.research.google.com/github/smthomas1704/restoration-rag/blob/main/chunk_from_GROBID_generated_TEI_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown==v4.6.3

!gdown https://drive.google.com/file/d/1U8mrDhhHdrLxQGn5TEIlemMV-46f9oLL/view?usp=drive_link -O /content/functional_trait_literature_grobid_tei_zipped.zip --fuzzy

!unzip /content/functional_trait_literature_grobid_tei_zipped.zip

### Processing TEI files.
1. We will be splitting the GROBID generated TEI files into smaller chunks.
2. Only paragraphs with titles and body will be used. Will skip references at the end of the page.
3. However, we do want to include the inline references to other material. Need to figure out how to do that.

### References:
1. https://kermitt2-grobid.hf.space/
2. https://python.langchain.com/docs/integrations/document_loaders/grobid
3. https://research.google.com/colaboratory/local-runtimes.html#:~:text=You%20can%20either%20run%20Jupyter,and%20the%20resource%20utilization%20monitor.
4. https://pypi.org/project/grobid-tei-xml/
5. https://stackoverflow.com/questions/2136267/beautiful-soup-and-extracting-a-div-and-its-contents-by-id
6. https://grobid.readthedocs.io/en/latest/Grobid-docker/

In [3]:
!pip install beautifulsoup4
!pip install lxml
!pip install pandas

In [69]:
from bs4 import BeautifulSoup

test_doc = '/content/funtional_trait_literature_grobid_tei/Carlucci et al. - 2020 - Functional traits and ecosystem services in ecolog.pdf.tei.xml'
with open(test_doc, 'r') as tei:
    soup = BeautifulSoup(tei, 'lxml')

# all_divs = soup.find_all("div")
# for a in all_divs:
#   print(a)

print(soup.get_text())

# # find all tables
# find_table = file.find('table', class_='numpy-table')
# rows = find_table.find_all('tr')

# # display tables
# for i in rows:
#     table_data = i.find_all('td')
#     data = [j.text for j in table_data]
#     print(data)






Functional traits and ecosystem services in ecological restoration

FAPEG


MCTIC/CNPq


Coordenação de Aperfeiçoamento de Pessoal de Nível Superior-Brasil (CAPES)


FAPESP



Wiley
Copyright Wiley

2020-11





MarcosBCarlucci
marcos.carlucci@gmail.com
0000-0002-5868-7090

Departamento de Botânica
Laboratório de Ecologia Funcional de Comunidades (LABEF)
Universidade Federal do Paraná

81531-980
Curitiba
Paraná
Brazil




PedroH SBrancalion
0000-0001-8245-4062

Departamento de Ciências Florestais
Escola Superior de Agricultura "Luiz de Queiroz
" Universidade de São Paulo

13418-900
Piracicaba, São Paulo
Brazil




RicardoRRodrigues
0000-0003-4818-0736

Departamento de Ciências Biológicas
Escola Superior de Agricultura "Luiz de Queiroz"
Universidade de São Paulo

13418-900
Piracicaba, São Paulo
Brazil




RafaelLoyola
0000-0001-5323-2735

Fundação Brasileira para o Desenvolvimento Sustentável

Rio de Janeiro
22610-180
Rio de Janeiro
Brazil



Departamento de Ecologia
Universidade Fe

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
